In [1]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample
from cmath import nan

In [80]:

batch_name='batch 1'

qualification_name_for_testin='UPDATE-eligible for next step memory rep'
qualification_name_for_entire_experiment='UPDATE-completed memory rep'

#all data location is relational to the location of this jupiter notebook:
batch_data_location=path.Path.cwd().parent
batch_data_location = batch_data_location / 'data' / 'pilot_24_hours' 


if (batch_data_location / batch_name).is_dir():
    batch_data_location= batch_data_location / batch_name
    print('Batch found',batch_data_location)
else: 
    print(f'PROBLEM FOUND: the requested batch name {batch_name} does not apear in the folder: {batch_data_location}')


PATH_TO_DATA = batch_data_location / 'data'
if not PATH_TO_DATA.exists():
    print('There is no data folder in the requested location',batch_data_location / 'data')
else: 
    print('Data folder found',batch_data_location / 'data')




Batch found c:\Users\d_abe\Desktop\human similarity database\edens github repo\Visual-memory-task\data\pilot_24_hours\batch 1
Data folder found c:\Users\d_abe\Desktop\human similarity database\edens github repo\Visual-memory-task\data\pilot_24_hours\batch 1\data


In [106]:
#paste the shelf here: 
shelf_dict={
  "A2KDZLWD8RAHDM": [
    999,
    999
  ],
  "AABZGLZ5QWCG0": [
    999,
    999,
    999
  ],
  "A2I6HVUDBTGD1": [
    999,
    999
  ],
  "A248QG4DPULP46": [
    2,
    1666796745860,
    1666875548743
  ],
  "ANGJEGCCP3AO2": [
    999,
    999
  ],
  "A2LHZMUV3Q81X0": [
    999,
    999
  ],
  "A3CFG3M2O22KYQ": [
    999,
    999
  ],
  "A33C0ZCZH9CZKA": [
    999,
    999
  ],
  "A1JJYY622DGE5L": [
    999,
    1666798289847
  ],
  "A2ASR7XQA7KERU": [
    999,
    1666798330693
  ],
  "A1618RA11ZOD8Y": [
    999,
    999
  ],
  "A1XMGLCK09Z0DW": [
    999,
    999,
    999
  ],
  "AAECD8RBWBLU9": [
    999,
    999,
    999
  ],
  "ACA2RL31B2G57": [
    999,
    999
  ],
  "AM27MWA9NUQQH": [
    999,
    999
  ],
  "A1D5QV7YAPIC30": [
    999,
    999,
    999
  ],
  "A1K4RESK1XUMQA": [
    999,
    999
  ],
  "AIYQVOLW7CF3Z": [
    999,
    999
  ],
  "A34J6WUTOV16M1": [
    999,
    999
  ],
  "A13THDIE8LTQ11": [
    999,
    999
  ],
  "A2RS15DB7NDYIF": [
    999,
    1666802624130
  ],
  "A32FHGV347AF6M": [
    999,
    999
  ],
  "A26F7OWQB4H0CH": [
    999,
    1666803549164
  ],
  "A22HIX1M4QXZBB": [
    19,
    1666804285762,
    1666882028114
  ],
  "A244N987STMI0V": [
    999,
    999
  ],
  "A1Q3GFQXJMIHQ8": [
    999,
    999,
    999
  ],
  "A36CTSMHOB5YN9": [
    999,
    999
  ],
  "A2J1DNVMJ56JG1": [
    22,
    1666806879557,
    1666887012627
  ],
  "A3EUH97BRHRSI6": [
    999,
    999
  ],
  "A2Y69684WT44XU": [
    999,
    1666808929934
  ]
}

In [81]:
#this section reads all the relevant files: 
if (batch_data_location / 'Batch_encoding_batch_results.csv').exists():
    encoding_df=pd.read_csv(batch_data_location / 'Batch_encoding_batch_results.csv')
else: 
    print(f'no "Batch_encoding_batch_results.csv" was found in data')

if (batch_data_location / 'Batch_testing_batch_results.csv').exists():
    testing_df=pd.read_csv(batch_data_location / 'Batch_testing_batch_results.csv')
else: 
    print(f'no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)')
if (batch_data_location / 'Batch_workers.csv').exists():
    workers_df=pd.read_csv(batch_data_location / 'Batch_workers.csv')
else:
    print('No AMAZON worker list was found: please make sure to upload one and name it correctly')



In [82]:
def find_largest_consequtive_repetition(responses_array):
#add a stop value so the loop will use the logic also if the end of the responses is all repeating: 
    responses_array=np.append(responses_array,[99])
    longest_rep_dict=dict()
    last_input=responses_array[0]
    consequtive_rep_counter=0
    for response in responses_array[1:]: 
        if response==last_input: #if consequtive rep: 
            consequtive_rep_counter=consequtive_rep_counter+1 #add to counter
        else: #once a new entry is in the vector - store the counter rep information 
            if last_input in longest_rep_dict.keys(): #if it was allready stored in our dictionary
                if longest_rep_dict[last_input]<consequtive_rep_counter: #replace the counter only if it is higher than what stored in dict
                    longest_rep_dict[last_input]=consequtive_rep_counter
            else: #this entry is not yet in the dictionary: add it 
                longest_rep_dict[last_input]=consequtive_rep_counter
            
            consequtive_rep_counter=0
            last_input=response
    return max(longest_rep_dict.values())

In [83]:
def process_worker_results(PATH_TO_BATCH,subject_name):
    cur_sub=pd.read_csv(PATH_TO_BATCH / subject_name)
    sub_demographics=cur_sub[['workID','Age','Gender']].iloc[0]

    #extract demo related information: 

    #if participant did not click on the arrow during the demo a 'demo_encoding_response.rt' colmumn wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('demo_encoding_response.rt' in cur_sub.columns):
        cur_sub['demo_encoding_response.rt']=np.nan
    demo_columns=['demo_encoding_loop.thisTrialN','DemoImage','DemoCorrect','demo_encoding_response.rt','demo_encoding_response.keys']

    sub_demo_information=cur_sub[demo_columns]
    empty_inds=sub_demo_information.loc[sub_demo_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_information=sub_demo_information.drop(index=empty_inds)


    #extract the demo test columns: 
    demo_test_columns=['demo_test_response.keys','demo_test_response.corr','demo_test_response.rt','demo_test_loop.thisTrialN','DemoImage1','DemoImage2','DemoCorrectTest']
    sub_demo_test_information=cur_sub[demo_test_columns]
    empty_inds=sub_demo_test_information.loc[sub_demo_test_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_test_information=sub_demo_test_information.drop(index=empty_inds)

    demo_df=pd.concat([sub_demo_information.reset_index(),sub_demo_test_information.reset_index()],axis=1)

    #extract real experiment related information: 

    #if participant did not click on the arrow during the encoding a 'test_encoding_response.rt' column wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('test_encoding_response.rt' in cur_sub.columns):
        cur_sub['test_encoding_response.rt']=np.nan


    encoding_related_columns=['test_encoding_response.keys','test_encoding_response.corr','trials.thisTrialN','target_image','pair','layer','correct','test_encoding_response.rt','key_resp_end.keys']
    sub_encoding_information=cur_sub[encoding_related_columns]
    #encoding section ends with a key press of the space key: so seperate this phase by finding this space key row
    end_of_section_ind=np.where(sub_encoding_information['key_resp_end.keys']=='space')[0][0]
    sub_encoding_information=sub_encoding_information.iloc[0:end_of_section_ind]
    #remove all the rows that precede the real encoding phase: 
    empty_inds=sub_encoding_information.loc[sub_encoding_information.isnull().apply(lambda x: all(x), axis=1)].index
    sub_encoding_information=sub_encoding_information.drop(index=empty_inds).reset_index()

    #extract real experiment TEST related information: 
    #test_related_columns=['layer','correct','test_test_response.keys','test_test_response.corr','test_test_response.rt','trials_2.thisRepN','trials_2.thisTrialN','trials_2.thisN','trials_2.thisIndex','trials_2.ran','image1','image2']
    #sub_test_information=cur_sub[test_related_columns].iloc[end_of_section_ind+2:-1]


    subject_dictionary=dict()
    subject_dictionary['demographics']=sub_demographics
    subject_dictionary['demo_df']=demo_df
    subject_dictionary['encoding_df']=sub_encoding_information
    #subject_dictionary['test_df']=sub_test_information

    return subject_dictionary



In [84]:
all_filenames=[file for file in PATH_TO_DATA.iterdir() if 'csv' in file.name and 'ENCODING' in file.name]
print('current csv files:',all_filenames)

qualification_for_test_df=pd.DataFrame(columns=['workerID','arrow_acc','mean_arrow_RT'])
for subject_csv in all_filenames:
    subject_dict=process_worker_results(PATH_TO_DATA,path.Path(subject_csv))
    cur_sub_encoding=subject_dict['encoding_df']

    sname=subject_csv.name.split('_')[1] #change according to actuall format. 

    #get only attention check related info: 
    cur_sub_encoding=cur_sub_encoding[['test_encoding_response.keys','correct','test_encoding_response.rt']].copy().dropna(how = 'all')
    if len(cur_sub_encoding)==0: #empty - the participant didnt respond on the arrow at all
        arrow_acc=0
        RT=nan
    else: 
        RT=cur_sub_encoding['test_encoding_response.rt'].mean()
        arrow_acc=(cur_sub_encoding['correct']==cur_sub_encoding['test_encoding_response.keys']).mean()
    sub_df=pd.DataFrame({'workerID':sname,'arrow_acc':arrow_acc,'mean_arrow_RT':RT},index=[sname])
    qualification_for_test_df=pd.concat([qualification_for_test_df,sub_df],axis=0)


current csv files: [WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 1/data/ENCODING_A1618RA11ZOD8Y_2022-10-26_11h33.42.790.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 1/data/ENCODING_A1JJYY622DGE5L_2022-10-26_11h31.29.847.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 1/data/ENCODING_A22HIX1M4QXZBB_2022-10-26_13h11.25.762.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 1/data/ENCODING_A244N987STMI0V_2022-10-26_13h16.54.309.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 1/data/ENCODING_A248QG4DPULP46_2022-10-26_11h05.45.860.csv'), WindowsPath('c:/Users/d_abe/Desktop/human sim

In [86]:
qualification_for_test_df['in_encoding_workers_list']=nan

In [87]:
for curr_worker_ID in qualification_for_test_df.index:
    if curr_worker_ID in encoding_df['WorkerId'].values:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(1)
    else:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(0)


In [100]:
qualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:]
qualified_df

,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A22HIX1M4QXZBB,A22HIX1M4QXZBB,1.0,0.69246,1.0
A248QG4DPULP46,A248QG4DPULP46,1.0,0.53968,1.0
A26F7OWQB4H0CH,A26F7OWQB4H0CH,0.6,0.16346,1.0
A2ASR7XQA7KERU,A2ASR7XQA7KERU,1.0,1.60104,1.0
A2J1DNVMJ56JG1,A2J1DNVMJ56JG1,0.8,1.296075,1.0
A2Y69684WT44XU,A2Y69684WT44XU,0.6,1.562533,1.0


In [101]:
disqualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) | (qualification_for_test_df['in_encoding_workers_list']==0),:]
disqualified_df

,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A1618RA11ZOD8Y,A1618RA11ZOD8Y,0.2,0.952,1.0
A1JJYY622DGE5L,A1JJYY622DGE5L,1.0,0.8464,0.0
A244N987STMI0V,A244N987STMI0V,0.4,0.368833,0.0
A2I6HVUDBTGD1,A2I6HVUDBTGD1,0.0,NaN,1.0
A2LHZMUV3Q81X0,A2LHZMUV3Q81X0,0.4,0.60784,0.0
A2RS15DB7NDYIF,A2RS15DB7NDYIF,0.6,2.137033,0.0


In [128]:
#update the workers list with the relevant qualification: 
#add the qualification for all eligible participants for the test session: 
ind_in_workers_df=[]
for worker_id in qualified_df.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin].index[0])
#remove from experiment all those that disqualified (so they wont be able to take the test session): 
for worker_id in disqualified_df.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment].index[0])

#print the two relevant columns of the updated workers_list: 
print('these are all participants that will be updated in the workers list:')
workers_df.loc[ind_in_workers_df,['Worker ID',qualification_name_for_testin,qualification_name_for_entire_experiment]]




these are all participants that will be updated in the workers list:


,Worker ID,UPDATE-eligible for next step memory rep,UPDATE-completed memory rep
554,A22HIX1M4QXZBB,1.0,NaN
585,A248QG4DPULP46,1.0,NaN
617,A26F7OWQB4H0CH,1.0,NaN
684,A2ASR7XQA7KERU,1.0,NaN
809,A2J1DNVMJ56JG1,1.0,NaN
1008,A2Y69684WT44XU,1.0,NaN
86,A1618RA11ZOD8Y,1.0,1.0
791,A2I6HVUDBTGD1,1.0,1.0


In [129]:
#save the worker list to upload (to update the qualifications)
workers_df.to_csv(batch_data_location/'Batch_workers_for_upload.csv',index=False)

In [130]:
for key in shelf_dict.keys():
    cur_entries=shelf_dict[key]
    if len(cur_entries)==2:
       cur_entries[0]=999

    shelf_dict


{'A2KDZLWD8RAHDM': [999, 999],
 'AABZGLZ5QWCG0': [999, 999, 999],
 'A2I6HVUDBTGD1': [999, 999],
 'A248QG4DPULP46': [2, 1666796745860, 1666875548743],
 'ANGJEGCCP3AO2': [999, 999],
 'A2LHZMUV3Q81X0': [999, 999],
 'A3CFG3M2O22KYQ': [999, 999],
 'A33C0ZCZH9CZKA': [999, 999],
 'A1JJYY622DGE5L': [999, 1666798289847],
 'A2ASR7XQA7KERU': [999, 1666798330693],
 'A1618RA11ZOD8Y': [999, 999],
 'A1XMGLCK09Z0DW': [999, 999, 999],
 'AAECD8RBWBLU9': [999, 999, 999],
 'ACA2RL31B2G57': [999, 999],
 'AM27MWA9NUQQH': [999, 999],
 'A1D5QV7YAPIC30': [999, 999, 999],
 'A1K4RESK1XUMQA': [999, 999],
 'AIYQVOLW7CF3Z': [999, 999],
 'A34J6WUTOV16M1': [999, 999],
 'A13THDIE8LTQ11': [999, 999],
 'A2RS15DB7NDYIF': [999, 1666802624130],
 'A32FHGV347AF6M': [999, 999],
 'A26F7OWQB4H0CH': [999, 1666803549164],
 'A22HIX1M4QXZBB': [19, 1666804285762, 1666882028114],
 'A244N987STMI0V': [999, 999],
 'A1Q3GFQXJMIHQ8': [999, 999, 999],
 'A36CTSMHOB5YN9': [999, 999],
 'A2J1DNVMJ56JG1': [22, 1666806879557, 1666887012627],
 'A